In [1]:
import sys
sys.path.append('../algorithms')
sys.path.append('../datasets')

import torch

import matplotlib.pyplot as plt

from datasets import HSIDataset
from grad_alignement import GradAlignement
from nabla import nabla

from torchvision import transforms
from math import sqrt


In [2]:
# Define device (default is "cpu")
device = "cuda"

# Define dtype
dtype = torch.float64

# Define random seed
seed = 42
torch.manual_seed(seed)

# Define data path
data_path = '/home/mhiriy/data/harvard.zarr'

# Signal to Noise Ratio
SNR = 10

In [3]:
# Load Datasets 
val_transform = transforms.Compose([transforms.ToTensor()]) # Transforms a the input data to torch tensors
dataset = HSIDataset(root_dir=data_path, split='train', transform=val_transform)

# Choose the index of the selected image in the dataset (ENMAP dataset contains 11 000 images)
idx = 43
x = dataset[idx].unsqueeze(0).to(device=device,dtype=dtype) # import image to device (cpu or gpu), sizes of x is [1,number of bands, width, height]
# Adds a small amount of white gaussian noise to the input HSI (sigma2 = 1e-4)
x += 1e-2*torch.rand_like(x, device=device, dtype=dtype)/torch.norm(x)

# Compute the panchromatic image from the ground truth HSI
panc = torch.sum(x, dim=1).unsqueeze(1)/x.shape[1]
grad_panc = nabla(panc)

# Adds noise to the input HSI
sigma2 = 10**(-SNR/10) * torch.norm(x, dim=[2,3])**2 / x.shape[2] / x.shape[3]
sigma2 = sigma2.unsqueeze(0).unsqueeze(1).reshape(1, sigma2.numel(), 1, 1)
sigma2 = sigma2.repeat(1, 1, x.shape[2], x.shape[3])

y = x + torch.sqrt(sigma2)*torch.randn_like(x, device=device, dtype=dtype)


In [ ]:
## Plot noisy and ground truth per band

cmap = 'gray'

band_index = [0, 10,20,30]

plt.figure(figsize=(40, 5))
for i, idx in enumerate(band_index):
    plt.subplot(151+i)
    plt.imshow(x[0, idx, ...].cpu().numpy(), cmap=cmap)
    plt.colorbar()
    plt.title('Ground Truth HSI - bands = {}'.format(idx))
    plt.axis('off')

plt.figure(figsize=(40, 5))
for i, idx in enumerate(band_index):
    plt.subplot(151+i)
    plt.imshow(y[0, idx, ...].cpu().numpy(), cmap=cmap)
    plt.colorbar()
    plt.title('Noisy HSI - band = {}'.format(idx))
    plt.axis('off')

plt.show()

In [ ]:

# Define the solver parameters
params = {}

params['compute_L'] = {'nband': x.shape[1]}
params['K'] = {}
params['K_adjoint'] = {}
params['prox_sigma_g_conj'] = {'alpha': grad_panc, 'eps': 1e-12}
params['prox_tau_f'] = {'y': y, 'sigma2': 1}
params['loss_fn'] = {'grad_panc': grad_panc, 'sigma2': 1}

sigma = 0.99
tau = 0.99
optim = GradAlignement(max_iter=1000, lmbda=1, theta=1, sigma=sigma, tau=tau)

x_est, loss = optim(y, init=None, verbose=False, params=params)

In [ ]:
## Show loss function

plt.figure()
plt.plot(loss)
plt.xlabel('Iterations')
plt.ylabel('Loss function')
plt.show()

In [ ]:
## Plot estimated, noisy and ground truth in RGB

def to_rgb(x):
    rgb_gt = x.cpu().numpy()[0, rgb_index, ...].transpose(1, 2, 0)
    rgb_gt = (rgb_gt - rgb_gt.min())/(rgb_gt.max()- rgb_gt.min())
    return rgb_gt

rgb_index = dataset.rgb_index

rgb_gt = to_rgb(x)
rgb_noisy = to_rgb(y)
rgb_estimated = to_rgb(x_est)

plt.figure(figsize=(10,30))

plt.subplot(131)
plt.imshow(rgb_noisy)
plt.title('Noisy Image')

plt.subplot(132)
plt.imshow(rgb_estimated)
plt.title('Estimated Image')

plt.subplot(133)
plt.imshow(rgb_gt)
plt.title('Ground truth Image')
plt.axis('off')


In [ ]:
## Plot estimated, noisy and ground truth per band

out = x_est
cmap = 'gray'

band_index = [0, 10,20,30]

plt.figure(figsize=(40, 5))
for i, idx in enumerate(band_index):
    plt.subplot(151+i)
    plt.imshow(x[0, idx, ...].cpu().numpy(), cmap=cmap)
    plt.colorbar()
    plt.title('Ground Truth HSI - bands = {}'.format(idx))
    plt.axis('off')

plt.figure(figsize=(40, 5))
for i, idx in enumerate(band_index):
    plt.subplot(151+i)
    plt.imshow(y[0, idx, ...].cpu().numpy(), cmap=cmap)
    plt.colorbar()
    plt.title('Noisy HSI - band = {}'.format(idx))
    plt.axis('off')

plt.figure(figsize=(40, 5))
for i, idx in enumerate(band_index):
    plt.subplot(151+i)
    plt.imshow(out[0, idx, ...].cpu().numpy(), cmap=cmap)
    plt.colorbar()
    plt.title('Denoised HSI - band = {}'.format(idx))
    plt.axis('off')

plt.show()

In [ ]:
## Plot eigenimages of estimated, noisy and ground truth per band

from datasets import get_eigenimages

z = get_eigenimages(x)
z_est = get_eigenimages(x_est)
z_noisy = get_eigenimages(y)

eigen_index = [1,3,5,7,9]

cmap = 'bone'

plt.figure(figsize=(30, 5))
for i, idx in enumerate(eigen_index):
    plt.subplot(151+i)
    plt.imshow(z[0, idx, ...].cpu().numpy(), cmap=cmap)
    plt.colorbar()
    plt.title('Eigenimage {}'.format(idx))
    plt.axis('off')

plt.figure(figsize=(30, 5))
for i, idx in enumerate(eigen_index):
    plt.subplot(151+i)
    plt.imshow(z_noisy[0, idx, ...].cpu().numpy(), cmap=cmap)
    plt.colorbar()
    plt.title('Noisy Eigenimage {}'.format(idx))
    plt.axis('off')

plt.figure(figsize=(30, 5))
for i, idx in enumerate(eigen_index):
    plt.subplot(151+i)
    plt.imshow(z_est[0, idx, ...].cpu().numpy(), cmap=cmap)
    plt.colorbar()
    plt.title('Denoised Eigenimage {}'.format(idx))
    plt.axis('off')

plt.show()